In [16]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('GROQ_API_KEY')


In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant", api_key=api_key)

In [18]:
llm.invoke('hello')

AIMessage(content='Hello. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 36, 'total_tokens': 54, 'completion_time': 0.024, 'prompt_time': 0.001688372, 'queue_time': 0.052900928, 'total_time': 0.025688372}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c6c4b913-677a-4b4d-add5-0f4cc0c659a6-0', usage_metadata={'input_tokens': 36, 'output_tokens': 18, 'total_tokens': 54})

In [19]:
# Create as State
class LLMState(TypedDict):
    question: str
    answer: str

In [20]:
def llm_qa(state: LLMState) -> LLMState:
    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # Ask that question to the LLM
    answer = llm.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [25]:
# Create the graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add_edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [27]:
# execute
initial_state = {'question': 'How far is moon from the earth?'}
final_state = workflow.invoke(initial_state)
print(final_state['answer'])

The average distance from the Earth to the Moon is approximately 384,400 kilometers (238,900 miles). However, this distance can vary slightly due to the elliptical shape of the Moon's orbit around the Earth. At its closest point (called perigee), the Moon is about 363,300 kilometers (225,300 miles) away, and at its farthest point (apogee), it is around 405,500 kilometers (252,000 miles) away.
